In [1]:
# Library Load Model
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Library Pre-Processing
from nltk.stem import WordNetLemmatizer
import nltk
import re
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SatriyaFzn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SatriyaFzn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SatriyaFzn\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# loading token
with open('t.pickle', 'rb') as file_1:
    t = pickle.load(file_1)

In [4]:
# Load Model
  
model_imp = load_model('best_model.h5')

In [5]:
model_imp.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1835, 128)         6913920   
                                                                 
 bidirectional (Bidirectiona  (None, 1835, 64)         41216     
 l)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 8)                 2336      
                                                                 
 dropout (Dropout)           (None, 8)                 0         
                                                                 
 dense (Dense)               (None, 1)                 9         
                                                                 
Total params: 6,957,481
Trainable params: 0
Non-trainable params: 6,957,481
______________________________________________

In [6]:
stopwords = stopwords.words('english')

In [15]:
lemmatizer = WordNetLemmatizer()

def text_processing(text):
    # Transforming "n't" into not
    text = text.replace("n't", "not")
    text = text.replace("nt", "not")

    # Converting all text to Lowercase
    text = text.lower()

    # Removing Unicode Characters
    text = re.sub(r'[^\x00-\x7F]', '', text)

    # Removing punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub("[^A-Za-z\s']", " ", text)
  
    # Menghilangkan \n
    text = re.sub(r"\\n", " ", text)

    # Removing Whitespace
    text = text.strip()

    # Removing double space
    text = re.sub("\s\s+" , " ", text)
        
    # Tokenizing words
    tokens = word_tokenize(text)

    # Removing Stopwords
    text = ' '.join([word for word in tokens if word not in stopwords])

    # Lemmatizer
    sentence = []
    for word in text.split():
        lem_word = lemmatizer.lemmatize(word, pos='v')  # Lemmatize verb
        lem_word = lemmatizer.lemmatize(lem_word, 'n')  # Lemmatize noun
        sentence.append(lem_word)
    
    text = ' '.join(sentence)

    return text


In [10]:
df_inf = pd.read_csv('hh8dsft_P2M2_Satriya_Fauzan_Adhim_Inference.csv')
df_inf_copy = df_inf.copy()
df_inf_copy = df_inf_copy.drop(['Unnamed: 0'], axis=1)
df_inf_copy['Rating'].replace({1: 'Negative Review', 2: 'Negative Review', 3: 'Negative Review', 4: 'Positive Review', 5: 'Positive Review'}, inplace=True)

In [11]:
df_inf_copy

,Review,Rating
0,fabulous resort perfect hotel neice celebrate ...,Positive Review
1,ehhh better punta cana twice compared hotel st...,Negative Review
2,nice hotel great location great place stay cer...,Positive Review
3,"grand oasis wonderful second time, group 20 fr...",Positive Review
4,not bad stay stayed hotel family attending jav...,Negative Review
5,disappointing expected 4* hotel.we arrived tol...,Negative Review
6,great place stay heart paris fantastic stay re...,Positive Review
7,"4 n't think, decided book atenea night stay de...",Negative Review
8,awesome time just returned vacation fantastic ...,Positive Review
9,superb stayed ubud village resort 3 nights hon...,Positive Review


In [16]:
# Applying all preprocessing in one document

df_inf_copy['Review_processed'] = df_inf_copy['Review'].apply(lambda x: text_processing(x))
df_inf_copy

,Review,Rating,Review_processed
0,fabulous resort perfect hotel neice celebrate ...,Positive Review,fabulous resort perfect hotel neice celebrate ...
1,ehhh better punta cana twice compared hotel st...,Negative Review,ehhh better punota cana twice compare hotel st...
2,nice hotel great location great place stay cer...,Positive Review,nice hotel great location great place stay cer...
3,"grand oasis wonderful second time, group 20 fr...",Positive Review,grand oasis wonderful second time group friend...
4,not bad stay stayed hotel family attending jav...,Negative Review,bad stay stay hotel family attend javaone lot ...
5,disappointing expected 4* hotel.we arrived tol...,Negative Review,disappoinoting expect hotelwe arrive tell room...
6,great place stay heart paris fantastic stay re...,Positive Review,great place stay heart paris fanotastic stay r...
7,"4 n't think, decided book atenea night stay de...",Negative Review,think decide book atenea night stay december r...
8,awesome time just returned vacation fantastic ...,Positive Review,awesome time return vacation fanotastic time w...
9,superb stayed ubud village resort 3 nights hon...,Positive Review,superb stay ubud village resort night honeymoo...


In [18]:
# Transform Inference-Set 
df_inf_transform = df_inf_copy.Review_processed

In [19]:
word_index = t.word_index

In [20]:
word_index

{'': 1,
 'hotel': 2,
 'room': 3,
 'stay': 4,
 'great': 5,
 'good': 6,
 'staff': 7,
 'night': 8,
 'day': 9,
 'time': 10,
 'nice': 11,
 'location': 12,
 'clean': 13,
 'service': 14,
 'walk': 15,
 'beach': 16,
 'breakfast': 17,
 'place': 18,
 'like': 19,
 'food': 20,
 'resort': 21,
 'get': 22,
 'pool': 23,
 'bed': 24,
 'really': 25,
 'area': 26,
 'say': 27,
 'book': 28,
 'friendly': 29,
 'people': 30,
 'small': 31,
 'bar': 32,
 'little': 33,
 'recommend': 34,
 'excellenot': 35,
 'bathroom': 36,
 'look': 37,
 'view': 38,
 'best': 39,
 'minute': 40,
 'make': 41,
 'helpful': 42,
 'price': 43,
 'trip': 44,
 'floor': 45,
 'restauranot': 46,
 'use': 47,
 'lot': 48,
 'need': 49,
 'come': 50,
 'water': 51,
 'think': 52,
 'check': 53,
 'take': 54,
 'restauranots': 55,
 'drink': 56,
 'review': 57,
 'beautiful': 58,
 'leave': 59,
 'eat': 60,
 'ask': 61,
 'wenot': 62,
 'thing': 63,
 'try': 64,
 'return': 65,
 'comfortable': 66,
 'desk': 67,
 'love': 68,
 'problem': 69,
 'right': 70,
 'way': 71,
 'go'

In [21]:
df_inf_transform = t.texts_to_sequences(df_inf_transform)

In [22]:
# Padding the dataset to a maximum review length in words

df_inf_transform = pad_sequences(df_inf_transform, maxlen=1835 ) # max len = 1835  words

In [23]:
df_inf_transform

array([[    0,     0,     0, ...,     4,   399,    21],
       [    0,     0,     0, ...,  4002,   451,    78],
       [    0,     0,     0, ...,   439,   764,    76],
       ...,
       [    0,     0,     0, ...,     4,     2, 11519],
       [    0,     0,     0, ..., 15846,   601,   189],
       [    0,     0,     0, ...,   498,     4,    21]])

In [24]:
# Predict using selected model
y_pred_inf = model_imp.predict(df_inf_transform)
y_pred_inf = np.where(y_pred_inf >= 0.5, 1, 0)
y_pred_inf

1/1 [==============================] - 3s 3s/step


array([[1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1]])

In [25]:
# Creating dataframe prediction
y_pred_inf = pd.DataFrame(y_pred_inf, columns=['Prediction'],index=df_inf_copy.index)

# Comparing the real target and prediction
model_inf = pd.concat([df_inf_copy['Rating'], y_pred_inf['Prediction']], axis=1).sort_index()
model_inf

,Rating,Prediction
0,Positive Review,1
1,Negative Review,0
2,Positive Review,1
3,Positive Review,1
4,Negative Review,0
5,Negative Review,1
6,Positive Review,1
7,Negative Review,0
8,Positive Review,1
9,Positive Review,1


In [27]:
model_inf.Prediction.replace({0:'Negative Review',1:'Positive Review'}, inplace=True)
model_inf

,Rating,Prediction
0,Positive Review,Positive Review
1,Negative Review,Negative Review
2,Positive Review,Positive Review
3,Positive Review,Positive Review
4,Negative Review,Negative Review
5,Negative Review,Positive Review
6,Positive Review,Positive Review
7,Negative Review,Negative Review
8,Positive Review,Positive Review
9,Positive Review,Positive Review


The model roughly made 90% accuracy on this data inference, it only got one wrong prediction.